In [1]:
import pandas as pd
import geopandas as gpd

In [38]:
df = pd.read_parquet('./data/newyork_newark_quality_measures_with_latlong.parquet')

In [39]:
df.head()

,PRVDR_NUM,FAC_NAME,CMS Overall Rating (Stars),"30-Day Unplanned Readmissions, Hospital Wide",HCAHPS Overall hospital rating (H_HSP_RATING_LINEAR_SCORE),CLABSI Standardized Infection Ratio,CAUTI Standardized Infection Ratio,Patient Safety and Adverse Events Composite (PSI-90),Early Delivery (PC-01),Exclusive Breast Milk Feeding (PC-05),...,Count of READM Measures Worse,READM Group Footnote,Pt Exp Group Measure Count,Count of Facility Pt Exp Measures,Pt Exp Group Footnote,TE Group Measure Count,Count of Facility TE Measures,TE Group Footnote,INTPTLAT,INTPTLONG
0,310025,CAREPOINT HEALTH - BAYONNE MEDICAL CENTER,2.0,16.3,81.0,1.560,0.441,1.17,None,None,...,2.0,,8.0,8.0,,12.0,8.0,,40.662495,-74.110201
1,314020,ESSEX COUNTY HOSPITAL CENTER,NaN,NaN,NaN,NaN,NaN,NaN,None,None,...,NaN,19,NaN,NaN,19,NaN,NaN,19,40.858017,-74.229784
2,310083,EAST ORANGE GENERAL HOSPITAL,1.0,17.0,76.0,0.000,NaN,1.11,None,None,...,1.0,,8.0,8.0,,12.0,5.0,,40.755733,-74.217917
3,310040,CAREPOINT HEALTH-HOBOKEN UNIVERSITY MEDICAL CE...,1.0,15.7,84.0,2.625,2.051,1.17,6%,None,...,3.0,,8.0,8.0,,12.0,6.0,,40.745251,-74.027926
4,310054,"HACKENSACK MERIDIAN HEALTH, MOUNTAINSIDE MEDICAL",2.0,15.5,83.0,0.292,1.304,0.81,3%,None,...,2.0,,8.0,8.0,,12.0,6.0,,40.813487,-74.219011


In [40]:
df.columns

Index(['PRVDR_NUM', 'FAC_NAME', 'CMS Overall Rating (Stars)',
       '30-Day Unplanned Readmissions, Hospital Wide',
       'HCAHPS Overall hospital rating (H_HSP_RATING_LINEAR_SCORE)',
       'CLABSI Standardized Infection Ratio',
       'CAUTI Standardized Infection Ratio',
       'Patient Safety and Adverse Events Composite (PSI-90)',
       'Early Delivery (PC-01)', 'Exclusive Breast Milk Feeding (PC-05)',
       'Appropriate care for severe sepsis and septic shock (SEP-1)',
       'Average (median) time patients spent in the emergency department before leaving from the visit (OP-18B)',
       'Safe Use of Opioids - Concurrent Prescribing (SAFE_USE_OPIOIDS)',
       'MRI Lumbar Spine for Low Back Pain (OP-8)',
       'Abdomen CT Use of Contrast Material (OP-10)',
       'Outpatients who got cardiac imaging stress tests before low-risk outpatient surgery (OP-13)',
       'Percent of patients receiving follow-up care within 30 days after hospitalization for mental illness (FUH-30)',


In [41]:
df = df.rename(columns={'INTPTLONG': 'longitude', 
           'INTPTLAT': 'latitude',
           'PRVDR_NUM': 'certification_number', 
           'FAC_NAME': 'name', 
           'Address': 'address',
           'Hospital Ownership': 'ownership', 
           '30-Day Unplanned Readmissions, Hospital Wide': 'readmission_rate'
})


In [43]:
# we don't need all those columns - only keep some
cols = ['longitude', 'latitude', 'certification_number', 'name', 'ownership', 'address', 'readmission_rate']
df_subset = df[cols]


In [49]:
df_subset = df_subset.dropna(subset=['longitude', 'latitude', 'readmission_rate'])

In [50]:
df_subset

,longitude,latitude,certification_number,name,ownership,address,readmission_rate
0,-74.110201,40.662495,310025,CAREPOINT HEALTH - BAYONNE MEDICAL CENTER,Voluntary non-profit - Private,29 EAST 29TH ST,16.3
2,-74.217917,40.755733,310083,EAST ORANGE GENERAL HOSPITAL,Proprietary,300 CENTRAL AVE,17.0
3,-74.027926,40.745251,310040,CAREPOINT HEALTH-HOBOKEN UNIVERSITY MEDICAL CE...,Voluntary non-profit - Church,308 WILLOW AVE,15.7
4,-74.219011,40.813487,310054,"HACKENSACK MERIDIAN HEALTH, MOUNTAINSIDE MEDICAL",Voluntary non-profit - Private,1 BAY AVENUE,15.5
5,-74.025027,40.793644,310003,PALISADES MEDICAL CENTER,Voluntary non-profit - Private,7600 RIVER RD,15.3
...,...,...,...,...,...,...,...
127,-73.296337,40.709005,330286,GOOD SAMARITAN HOSPITAL MEDICAL CENTER,Voluntary non-profit - Church,1000 MONTAUK HIGHWAY,15.9
128,-73.066659,40.956068,330185,JOHN T MATHER MEMORIAL HOSPITAL OF PORT JEFFE...,Voluntary non-profit - Private,75 NORTH COUNTRY ROAD,13.9
133,-72.987407,40.761780,330141,LONG ISLAND COMMUNITY HOSPITAL,Voluntary non-profit - Private,101 HOSPITAL ROAD,15.1
134,-74.221768,40.008534,310041,COMMUNITY MEDICAL CENTER,Voluntary non-profit - Private,99 RT 37 WEST,15.8


In [51]:
gdf = gpd.GeoDataFrame(
    df_subset[cols[2:]], geometry=gpd.points_from_xy(df_subset['longitude'], df_subset['latitude'])
    )

In [52]:
gdf

,certification_number,name,ownership,address,readmission_rate,geometry
0,310025,CAREPOINT HEALTH - BAYONNE MEDICAL CENTER,Voluntary non-profit - Private,29 EAST 29TH ST,16.3,POINT (-74.11020 40.66249)
2,310083,EAST ORANGE GENERAL HOSPITAL,Proprietary,300 CENTRAL AVE,17.0,POINT (-74.21792 40.75573)
3,310040,CAREPOINT HEALTH-HOBOKEN UNIVERSITY MEDICAL CE...,Voluntary non-profit - Church,308 WILLOW AVE,15.7,POINT (-74.02793 40.74525)
4,310054,"HACKENSACK MERIDIAN HEALTH, MOUNTAINSIDE MEDICAL",Voluntary non-profit - Private,1 BAY AVENUE,15.5,POINT (-74.21901 40.81349)
5,310003,PALISADES MEDICAL CENTER,Voluntary non-profit - Private,7600 RIVER RD,15.3,POINT (-74.02503 40.79364)
...,...,...,...,...,...,...
127,330286,GOOD SAMARITAN HOSPITAL MEDICAL CENTER,Voluntary non-profit - Church,1000 MONTAUK HIGHWAY,15.9,POINT (-73.29634 40.70900)
128,330185,JOHN T MATHER MEMORIAL HOSPITAL OF PORT JEFFE...,Voluntary non-profit - Private,75 NORTH COUNTRY ROAD,13.9,POINT (-73.06666 40.95607)
133,330141,LONG ISLAND COMMUNITY HOSPITAL,Voluntary non-profit - Private,101 HOSPITAL ROAD,15.1,POINT (-72.98741 40.76178)
134,310041,COMMUNITY MEDICAL CENTER,Voluntary non-profit - Private,99 RT 37 WEST,15.8,POINT (-74.22177 40.00853)


In [53]:
gdf.to_file('./data/newyork_hospitals.geojson', driver="GeoJSON")